In [ ]:
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np

Скачивание файлов из Яндекс диска

In [ ]:
def load_yandex_disk (public_key):
  '''
  Скачивание файлов из яндекс диска
  Данная функция принимает на вход ссылку на файл с Яндекс диска
  в виде текстовой строки (в кавычках)
  Пример 'https://disk.yandex.ru/d/bzrCg3jECpnF3Q'
  Перед использованием необходимо загрузить следующие библиотеки 
  import requests
  from urllib import parse
  На выходе
  Загружает файл в Phyton и возвращает название файла,
  Пример использования:
  df=pd.read_csv(load_yandex_disk ('ссылка'))

  '''
  base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

  # Получаем загрузочную ссылку
  final_url = base_url + parse.urlencode(dict(public_key=public_key))
  response = requests.get(final_url)
  download_url = response.json()['href']

  #Получаем имя файла из ссылки
  file_name = download_url.split('/')[6]
  file_name = file_name.split('&')[1]
  file_name = file_name.split('=')[1]
  file_name = parse.unquote(file_name)

  # Загружаем файл и сохраняем его
  download_response = requests.get(download_url)
  with open(file_name, 'wb') as f:
    f.write(download_response.content)
  return file_name

Функция,  поиска по сайту habr.com.

In [ ]:
def get_posts_from_blog_full(requests_habr, pages):
    '''
    Функция забирает материалы поиска с сайта habr.com
    и формирует датафрейм вида
    <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
    Принимает на вход 2 аргумента (поисковый запрос в виде списка запросов, количество страниц (с сайта) в поиске)
    '''
    
    habr_blog_full = pd.DataFrame()    
    for request in requests_habr:
      for page in range(1,pages +1):
        url = 'https://habr.com/ru/search/page' + str(page)
        params = { 'q': request}
        res = requests.post(url, params=params)
        time.sleep(0.33)
        soup = BeautifulSoup(res.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')

        for article in articles:
          date = article.find('span', class_='tm-article-snippet__datetime-published').text
          title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
          link = 'https://www.habr.com'+ article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').find('a').get('href')
        
          # Данный блок кода переходит по ссылке и забирает оттуда текст
          res_text = requests.get(link).text
          time.sleep(0.33)        
          soup_text = BeautifulSoup(res_text)
          articles_text = soup_text.find('div', class_='article-formatted-body')

          likes = article.find('svg', class_='tm-svg-img tm-votes-meter__icon tm-votes-meter__icon tm-votes-meter__icon_appearance-article').text

          row = {'date': date, 'title': title, 'link': link, 'text' : articles_text, 'likes' : likes}
          habr_blog_full = pd.concat([habr_blog_full, pd.DataFrame([row])])
    return habr_blog_full.reset_index(drop=True).drop_duplicates()

In [ ]:
get_posts_from_blog_full(['phyton', 'java'], 4)

Смоделируем игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш.

777: 200 руб.

999: 100 руб.

555: 50 руб.

333: 15 руб.

111: 10 руб.

*77: 5 руб.

**7: 3 руб.

*00: 2 руб.

**0: 1 руб.

Вопросы:

Выгодна ли игра игроку? Сколько в среднем приобретает или теряет игрок за одну игру?

In [ ]:
trials = 1000000
balans = 0

for i in range(trials):
    random_number = np.random.randint(0, 1000)    # Случайное число от 0 до 999
    if random_number == 777:
        balans += 199
    elif random_number == 999:
      balans += 99
    elif random_number == 555:
      balans += 49
    elif random_number == 333:
      balans += 14
    elif random_number == 111:
      balans += 9
    elif random_number % 100 == 77:
      balans += 4
    elif random_number % 10 == 7:
      balans += 2
    elif random_number % 100 == 0:
      balans += 1
    elif random_number % 10 == 0:
      balans += 0
    else:
      balans -= 1

if balans > 0:
  print ('При данных случайных числах игра выгодна игроку')
  print (f'В среднем игрок за одну игру приобретает {balans/trials} руб.')
elif balans < 0:
  print ('При данных случайных числах игра невыгодна игроку')
  print (f'В среднем игрок за одну игру теряет {abs(balans/trials)} руб.') 
else:
  print ('При данных случайных числах игрок ни чего не приобрел, но и не потерял')
print(f'Баланс игрока {balans} руб.')